## Predict Student Performance from Game Play feature preprocessing


In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_column', 50)

### Reading the train data logs

In [2]:
FOLDERPATH = "./data/"

In [3]:
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group' : str}

df_train = pd.read_csv(FOLDERPATH + "train.csv", dtype=dtypes)
print(f"The train dataset contains {df_train.shape[0]} rows and {df_train.shape[1]} columns")

df_train.head(10)

The train dataset contains 26296946 rows and 20 columns


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991394,-159.314682,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991394,-159.314682,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
5,20090312431273200,5,3423,person_click,basic,0,NaN,-412.991394,-157.314682,381.0,492.0,NaN,"Sure thing, Jo. Grab your notebook and come up...",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
6,20090312431273200,6,5197,person_click,basic,0,NaN,478.485077,-199.971680,593.0,485.0,NaN,"See you later, Teddy.",teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
7,20090312431273200,7,6180,person_click,basic,0,NaN,503.355133,-168.619919,609.0,453.0,NaN,I get to go to Gramps's meeting!,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
8,20090312431273200,8,7014,person_click,basic,0,NaN,510.733429,-157.720642,615.0,442.0,NaN,Now where did I put my notebook?,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4
9,20090312431273200,9,7946,person_click,basic,0,NaN,512.048035,-153.743637,616.0,438.0,NaN,\u00f0\u0178\u02dc\u00b4,teddy,tunic.historicalsociety.closet,tunic.historicalsociety.closet.teddy.intro_0_cs_0,0,0,1,0-4


In [4]:
CATEGORICAL_COLS = ['event_name', 'name','level', 'page', "fullscreen", "hq", "music", 'level_group']
ID_CATEGORICAL_COLS = ['fqid', 'room_fqid', 'text_fqid']

NUMERICAL_COLS = ['elapsed_time', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration']

In [5]:
def describe_column(df, column_name, numeric=True):
    # Calculate statistics
    num_nans = df[column_name].isna().sum()
    num_nulls = df[column_name].isnull().sum()
    num_unique = df[column_name].nunique()
    count = (df[column_name].count() / df[column_name].shape[0])*100
    if numeric:
        
        mean = df[column_name].mean()
        min_val = df[column_name].min()
        max_val = df[column_name].max()
        median = df[column_name].median()
        # Print statistics
        print(f"{column_name}:\n\tAvaiable: {count:.2f}%, Mean: {mean:.2f}, Min: {min_val}, Max: {max_val}, Median: {median:.2f}, Number of NaNs: {num_nans}, Number of nulls: {num_nulls},Number of unique values: {num_unique}")
    else:
        print(f"{column_name}:\n\tAvaiable: {count:.2f}%, Number of NaNs: {num_nans}, Number of nulls: {num_nulls}, Number of unique values: {num_unique}")

#### Feature analysis

In [6]:
print("Categorical column descriptions:")
for c_col in CATEGORICAL_COLS:
    describe_column(df_train, c_col, numeric=False)
    
print("ID Categorical column descriptions:")
for c_col in ID_CATEGORICAL_COLS:
    describe_column(df_train, c_col, numeric=False)

Categorical column descriptions:
event_name:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 11
name:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 6
level:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 23
page:
	Avaiable: 2.15%, Number of NaNs: 25732402, Number of nulls: 25732402, Number of unique values: 7
fullscreen:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 2
hq:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 2
music:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 2
level_group:
	Avaiable: 100.00%, Number of NaNs: 0, Number of nulls: 0, Number of unique values: 3
ID Categorical column descriptions:
fqid:
	Avaiable: 68.53%, Number of NaNs: 8274415, Number of nulls: 8274415, Number of unique values: 128
room_fqid:
	Avaiable: 100.00%, Number of NaNs: 0, Number 

### Reading the train labels

In [7]:
train_labels = pd.read_csv(FOLDERPATH + "train_labels.csv")
print(f"Loading {train_labels.shape[0]} labels.")

Loading 424116 labels.


In [8]:
print(f"Preprocessing session IDs...")

train_labels['session'] = train_labels.session_id.apply(lambda x: int(x.split('_')[0]) )
train_labels['q'] = train_labels.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
train_labels.head(5)

Preprocessing session IDs...


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [9]:
print(f"Labels contains {train_labels['session'].nunique()} sessions with answers to {train_labels['q'].nunique()} questions which is overall {train_labels.shape[0]} rows.")

Labels contains 23562 sessions with answers to 18 questions which is overall 424116 rows.


### basic prepocessing with aggregating logs

In [23]:
def feature_engineer(dataset_df):
    dfs = []
    
    for c in CATEGORICAL_COLS:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in ID_CATEGORICAL_COLS:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    
    for c in NUMERICAL_COLS:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    
    for c in NUMERICAL_COLS:
        tmp = dataset_df.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    dataset_df = pd.concat(dfs,axis=1)
    dataset_df = dataset_df.fillna(-1)
    dataset_df = dataset_df.reset_index()
    dataset_df = dataset_df.set_index('session_id')
    return dataset_df

In [24]:
prepocessed_df = feature_engineer(df_train)
print("Full prepared dataset shape is {}".format(prepocessed_df.shape))

Full prepared dataset shape is (70686, 24)


In [25]:
prepocessed_df.columns

Index(['level_group', 'event_name_nunique', 'name_nunique', 'level_nunique',
       'page_nunique', 'fullscreen_nunique', 'hq_nunique', 'music_nunique',
       'level_group_nunique', 'fqid_nunique', 'room_fqid_nunique',
       'text_fqid_nunique', 'elapsed_time', 'room_coor_x', 'room_coor_y',
       'screen_coor_x', 'screen_coor_y', 'hover_duration', 'elapsed_time_std',
       'room_coor_x_std', 'room_coor_y_std', 'screen_coor_x_std',
       'screen_coor_y_std', 'hover_duration_std'],
      dtype='object')

In [26]:
prepocessed_df.head(10)

,level_group,event_name_nunique,name_nunique,level_nunique,page_nunique,fullscreen_nunique,hq_nunique,music_nunique,level_group_nunique,fqid_nunique,...,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,elapsed_time_std,room_coor_x_std,room_coor_y_std,screen_coor_x_std,screen_coor_y_std,hover_duration_std
session_id,,,,,,,,,,,,,,,,,,,,,
20090312431273200,0-4,10,3,5,0,1,1,1,1,30,...,-71.413750,448.410248,383.044861,2389.500000,49246.539458,399.296051,129.292404,214.871002,104.082741,3227.370757
20090312431273200,13-22,10,3,10,0,1,1,1,1,49,...,-162.004303,442.489807,379.301025,899.925903,126666.129584,622.061401,230.370880,240.280212,99.067863,1305.088265
20090312431273200,5-12,10,3,8,0,1,1,1,1,39,...,-57.269321,451.950958,378.784912,969.333313,80175.676658,357.227692,137.409470,203.268555,120.255455,1316.408315
20090312433251036,0-4,11,4,5,1,1,1,1,1,22,...,-53.671082,358.223083,370.723083,1378.750000,67372.714092,445.980042,156.186249,252.554703,121.062927,2114.876406
20090312433251036,13-22,11,6,10,3,1,1,1,1,73,...,-142.861893,462.852478,387.930084,720.384949,777382.529186,529.575684,234.279587,259.288849,133.345688,1990.705518
20090312433251036,5-12,11,4,8,2,1,1,1,1,45,...,-53.836208,470.819275,375.771088,824.096802,159319.587112,377.074066,160.557098,232.280914,132.195572,1836.236232
20090312455206810,0-4,9,3,5,0,1,1,1,1,22,...,-31.125036,373.365509,481.268951,3145.666748,121848.824074,478.652405,161.438385,459.750366,177.665588,4575.148668
20090312455206810,13-22,11,4,10,3,1,1,1,1,47,...,-142.360870,639.000000,521.446716,1055.319946,125153.779971,642.936279,243.042694,395.599976,181.509079,1403.679502
20090312455206810,5-12,11,4,8,2,1,1,1,1,41,...,-51.599346,714.834839,510.330261,783.038452,76156.386207,363.422516,180.862289,346.987579,228.421677,825.029526


#### count when the user clicked on the wrong answer

In [65]:
# src: https://www.kaggle.com/code/janmpia/clues-clicks-eda-features#Features
out = True

right_tunic = ~((df_train.room_coor_x < 200) & (df_train.room_coor_x > 50 ) & (df_train.room_coor_y < 350) & (df_train.room_coor_y >200))
right_plaque  = ~((df_train.room_coor_x < 590) & (df_train.room_coor_x > 500 ) & (df_train.room_coor_y < -80) & (df_train.room_coor_y >-150))
right_businesscards  = ~((df_train.room_coor_x < 150) & (df_train.room_coor_x > 50 ) & (df_train.room_coor_y < -100) & (df_train.room_coor_y >-150))
right_logbook = ~((df_train.room_coor_y < 40) & (df_train.room_coor_y >-10))
right_reader = ~((df_train.room_coor_x < -150) & (df_train.room_coor_x > -300 ) & (df_train.room_coor_y < -90) & (df_train.room_coor_y >-120))
right_directory = ~((df_train.room_coor_x < -60) & (df_train.room_coor_x > -390 ) & (df_train.room_coor_y < -280) & (df_train.room_coor_y >-460))
right_tracks = ~((df_train.room_coor_x < 1100) & (df_train.room_coor_x > 950 ) & (df_train.room_coor_y < -320) & (df_train.room_coor_y >-500))
right_colorbook = True
right_reader_flag = ~((df_train.room_coor_x < 0) & (df_train.room_coor_x > -290 ) & (df_train.room_coor_y < 110) & (df_train.room_coor_y >-90))
right_journals_flag = True

FQID = ['tunic', 'plaque','businesscards','logbook','reader','tracks','colorbook','reader_flag','journals_flag']
RIGHT_FQID =  [right_tunic, right_plaque, right_businesscards, right_logbook, right_reader, right_tracks, right_colorbook, right_reader_flag, right_journals_flag]

In [73]:
df_clues = pd.DataFrame(df_train.groupby(['session_id','level_group'])['index'].count()).drop(columns = 'index')

In [74]:
def create_feature_from_clicks(fqid, right,lots = 10):
    IDX = ((df_train.fqid==fqid))&(df_train.event_name!='navigate_click') & out & right
    tmp = df_train.loc[IDX]
    events_in_room = tmp.groupby(['session_id','level_group'])['index'].count()
    return events_in_room

for fqid, right in zip(FQID, RIGHT_FQID):
    print(f"Counting {fqid} mistakes...")
    df_clues[f"{fqid}_mistakes"] = create_feature_from_clicks(fqid, right)

df_clues.reset_index(inplace=True)
df_clues = df_clues.fillna(0)
# Convert all columns except the first one to integer
mistake_cols = df_clues.columns[2:].to_list()
df_clues[mistake_cols] = df_clues[mistake_cols].astype(int)

df_clues.head(10)

Counting tunic mistakes...
Counting plaque mistakes...
Counting businesscards mistakes...
Counting logbook mistakes...
Counting reader mistakes...
Counting tracks mistakes...
Counting colorbook mistakes...
Counting reader_flag mistakes...
Counting journals_flag mistakes...


,session_id,level_group,tunic_mistakes,plaque_mistakes,businesscards_mistakes,logbook_mistakes,reader_mistakes,tracks_mistakes,colorbook_mistakes,reader_flag_mistakes,journals_flag_mistakes
0,20090312431273200,0-4,3,1,0,0,0,0,0,0,0
1,20090312431273200,13-22,0,0,0,0,0,2,4,3,1
2,20090312431273200,5-12,0,0,2,5,4,0,0,0,0
3,20090312433251036,0-4,1,9,0,0,0,0,0,0,0
4,20090312433251036,13-22,0,2,3,8,3,4,7,4,8
5,20090312433251036,5-12,0,0,2,3,23,0,0,0,0
6,20090312455206810,0-4,1,1,0,0,0,0,0,0,0
7,20090312455206810,13-22,0,0,0,0,0,2,1,2,1
8,20090312455206810,5-12,0,0,2,2,1,0,0,0,0
9,20090313091715820,0-4,3,6,0,0,0,0,0,0,0


In [75]:
prepocessed_df_w_clues = pd.merge(prepocessed_df, df_clues, on=['session_id', 'level_group'], how='left')
prepocessed_df_w_clues.head()

,session_id,level_group,event_name_nunique,name_nunique,level_nunique,page_nunique,fullscreen_nunique,hq_nunique,music_nunique,level_group_nunique,fqid_nunique,room_fqid_nunique,text_fqid_nunique,elapsed_time,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,elapsed_time_std,room_coor_x_std,room_coor_y_std,screen_coor_x_std,screen_coor_y_std,hover_duration_std,tunic_mistakes,plaque_mistakes,businesscards_mistakes,logbook_mistakes,reader_mistakes,tracks_mistakes,colorbook_mistakes,reader_flag_mistakes,journals_flag_mistakes
0,20090312431273200,0-4,10,3,5,0,1,1,1,1,30,7,17,8.579356e+04,7.701275,-71.413750,448.410248,383.044861,2389.500000,49246.539458,399.296051,129.292404,214.871002,104.082741,3227.370757,3,1,0,0,0,0,0,0,0
1,20090312431273200,13-22,10,3,10,0,1,1,1,1,49,12,35,1.040601e+06,-130.347168,-162.004303,442.489807,379.301025,899.925903,126666.129584,622.061401,230.370880,240.280212,99.067863,1305.088265,0,0,0,0,0,2,4,3,1
2,20090312431273200,5-12,10,3,8,0,1,1,1,1,39,11,24,3.572052e+05,14.306062,-57.269321,451.950958,378.784912,969.333313,80175.676658,357.227692,137.409470,203.268555,120.255455,1316.408315,0,0,2,5,4,0,0,0,0
3,20090312433251036,0-4,11,4,5,1,1,1,1,1,22,6,11,9.763342e+04,-84.045959,-53.671082,358.223083,370.723083,1378.750000,67372.714092,445.980042,156.186249,252.554703,121.062927,2114.876406,1,9,0,0,0,0,0,0,0
4,20090312433251036,13-22,11,6,10,3,1,1,1,1,73,16,43,2.498852e+06,-30.762283,-142.861893,462.852478,387.930084,720.384949,777382.529186,529.575684,234.279587,259.288849,133.345688,1990.705518,0,2,3,8,3,4,7,4,8
